In [187]:
from pyspark.sql import SparkSession, functions as F

In [188]:
spark = SparkSession.builder \
        .appName("JSON to CSV") \
        .getOrCreate()

In [189]:
df_exportacao = spark.read.json("../data/exportacao.json")
df_producao = spark.read.json("../data/producao.json")

In [190]:
df_exportacao.show()

+----+--------------+----------+--------------------+-------------+--------+
| ANO|     CATEGORIA|    ORIGEM|              PAISES|QUANTIDADE_KG|VALOR_US|
+----+--------------+----------+--------------------+-------------+--------+
|1970|Vinhos de mesa|EXPORTACAO|         Afeganistão|          0.0|     0.0|
|1970|Vinhos de mesa|EXPORTACAO|       África do Sul|          0.0|     0.0|
|1970|Vinhos de mesa|EXPORTACAO|Alemanha, Repúbli...|          0.0|     0.0|
|1970|Vinhos de mesa|EXPORTACAO|              Angola|          0.0|     0.0|
|1970|Vinhos de mesa|EXPORTACAO|            Anguilla|          0.0|     0.0|
|1970|Vinhos de mesa|EXPORTACAO|   Antígua e Barbuda|          0.0|     0.0|
|1970|Vinhos de mesa|EXPORTACAO| Antilhas Holandesas|        280.0|   207.0|
|1970|Vinhos de mesa|EXPORTACAO|      Arábia Saudita|          0.0|     0.0|
|1970|Vinhos de mesa|EXPORTACAO|           Argentina|          0.0|     0.0|
|1970|Vinhos de mesa|EXPORTACAO|               Aruba|          0.0|     0.0|

In [191]:
df_exportacao = df_exportacao.select("PAISES", "QUANTIDADE_KG", "VALOR_US").distinct() \
    .groupBy("PAISES") \
    .agg(
        F.sum("QUANTIDADE_KG").alias("Quantidade (kg)"),
        F.sum("VALOR_US").alias("Valor (US$)")
    ) \
    .withColumnRenamed("PAISES", "Pais")
df_exportacao.show()


+--------------------+---------------+-----------+
|                Pais|Quantidade (kg)|Valor (US$)|
+--------------------+---------------+-----------+
|          Luxemburgo|        95225.0|   489198.0|
|              Suécia|    1.3765481E7|2.1117428E7|
|            Anguilla|          454.0|      186.0|
|Provisão de Navio...|           30.0|       55.0|
|         Cook, Ilhas|          116.0|      839.0|
|  Bósnia-Herzegovina|          223.0|      266.0|
|              Canadá|    2.6724132E7| 5.126297E7|
|            Marrocos|           38.0|      122.0|
|            Jordânia|          618.0|      817.0|
|Coreia, Republica...|         5002.0|    20300.0|
|             Senegal|        45413.0|    45078.0|
|           Cingapura|       247197.0|   582396.0|
|          Cabo Verde|        45915.0|    82185.0|
|             Belgica|       134600.0|   646070.0|
|             Polônia|       243342.0|   813349.0|
|São Cristóvão e N...|           52.0|      183.0|
|  Trinidade e Tobago|       99

In [192]:
df_producao.show()

+----+--------------------+--------+--------------------+------------+
| ANO|           CATEGORIA|  ORIGEM|             PRODUTO|QUANTIDADE_L|
+----+--------------------+--------+--------------------+------------+
|1970|       VINHO DE MESA|PRODUCAO|               Tinto|1.74224052E8|
|1970|       VINHO DE MESA|PRODUCAO|              Branco|    748400.0|
|1970|       VINHO DE MESA|PRODUCAO|              Rosado| 4.2236152E7|
|1970|VINHO FINO DE MES...|PRODUCAO|               Tinto|   7591557.0|
|1970|VINHO FINO DE MES...|PRODUCAO|              Branco| 1.5562889E7|
|1970|VINHO FINO DE MES...|PRODUCAO|              Rosado|    744900.0|
|1970|                SUCO|PRODUCAO|Suco de uva integral|   1097771.0|
|1970|                SUCO|PRODUCAO|Suco de uva conce...|         0.0|
|1970|                SUCO|PRODUCAO| Suco de uva adoçado|         0.0|
|1970|                SUCO|PRODUCAO|Suco de uva orgânico|         0.0|
|1970|                SUCO|PRODUCAO|Suco de uva recon...|         0.0|
|1970|

In [193]:
df_producao = df_producao.select("PRODUTO", "QUANTIDADE_L").distinct() \
    .withColumnRenamed("PRODUTO", "produto") \
    .withColumnRenamed("QUANTIDADE_L", "quantidade (Litros)") \
    .groupBy("produto") \
    .agg(
        F.sum("quantidade (Litros)").alias("Total_quantidade (Litros)")
        )

df_producao.show()

+--------------------+-------------------------+
|             produto|Total_quantidade (Litros)|
+--------------------+-------------------------+
|Mosto parcialment...|                 543510.0|
|      Vinho Composto|                 163000.0|
|           Destilado|                2342496.0|
|Base espumante mo...|              4.1454618E7|
|      Base espumante|                6348269.0|
|  Espumante moscatel|                7525544.0|
|   Vinho acidificado|                  67500.0|
|           Bagaceira|                8672865.0|
|       Borra líquida|                9.39263E7|
|           Licorosos|                2532326.0|
|Mosto de uva com ...|                6160330.0|
|            Frisante|                3649759.0|
|Suco de uva orgânico|                2394539.0|
|      Vinho orgânico|              4.0209375E7|
|Suco de uva integral|             7.08432016E8|
|               Tinto|            8.876745662E9|
|               Pisco|                   4372.0|
|      Vinho licoros